<a href="https://colab.research.google.com/github/MJ199999/SW-Project/blob/master/ko_keybert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://wikidocs.net/159468



In [1]:
!pip install sentence_transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.3 MB/s 
     |████████████████████████████████| 4.7 MB 45.5 MB/s 
     |████████████████████████████████| 1.3 MB 71.7 MB/s 
     |████████████████████████████████| 120 kB 71.7 MB/s 
     |████████████████████████████████| 6.6 MB 48.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4c095c603f4f14c72fb685eb799417c6a4069a23ef3907c1af0e10c31823a500
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 5.8 MB/s 
     |████████████████████████████████| 453 kB 71.4 MB/s 


# 데이터 로드

In [6]:
import numpy as np
import itertools
import pandas as pd

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [31]:
df = pd.read_excel('/gdrive/MyDrive/Colab Notebooks/sw-project/SW-Project/poem_key.xlsx')

In [26]:
df.head()

,시인,제목,시,키워드
0,강은교,우리가 물이 되어,우리가 물이 되어 만난다면\n가문 어느 집에선들 좋아하지 않으랴.\n우리가 키 큰 ...,그리움
1,김춘수,꽃을 위한 서시,나는 시방 위험한 짐승이다.\n나의 손이 닿으면 너는 \n미지의 까마득한 어둠이 된...,그리움
2,김춘수,꽃,내가 그의 이름을 불러 주기 전에는\n그는 다만\n하나의 몸짓에 지나지 않았다. \...,그리움
3,김춘수,능금,그는 그리움에 산다.\n그리움은 익어서 \n스스로도 견디기 어려운\n빛깔이 되고 향...,그리움
4,김춘수,부두에서,바다에 굽힌 사나이들\n하루의 노동을 끝낸\n저 사나이들의 억센 팔에 안긴\n깨지지...,노동


In [45]:
for i in range(0, len(df)):
  poem = df['시'].iloc[i]
  okt = Okt()

  tokenized_doc = okt.pos(poem)
  tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
  
  
  n_gram_range = (1, 1)

  count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
  candidates = count.get_feature_names_out()


  model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
  doc_embedding = model.encode([poem])
  candidate_embeddings = model.encode(candidates)

In [48]:
type(df1)

str

In [46]:
df1 = df1.astype("string")

AttributeError: ignored

In [47]:
s = df1.to_string()

AttributeError: ignored

In [49]:
okt = Okt()

tokenized_doc = okt.pos(poem)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('우리', 'Noun'), ('가', 'Josa'), ('물이', 'Noun'), ('되어', 'Verb'), ('만난다면', 'Verb'), ('\n', 'Foreign'), ('가문', 'Noun'), ('어느', 'Adverb'), ('집', 'Noun'), ('에선', 'Josa')]
명사 추출 : 우리 물이 가문 집 우리 키 나무 서서 우르르 우르르 소리 녘 저 혼자 강물 나무 뿌리 기도 아아 처녀 바다 지금 우리 불로 벌써 숯 뼈 하나 세상 것 만 리 밖 그대 저 불 지난 뒤 물 불 소리 말 때 인적 하늘 오라


In [71]:
n_gram_range = (1, 1)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 24
trigram 다섯개만 출력 : ['가문' '강물' '그대' '기도' '나무']


In [72]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([df1])
candidate_embeddings = model.encode(candidates)

In [73]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['나무', '가문', '불로', '뿌리', '강물']


In [74]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [75]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['처녀', '기도', '나무', '가문', '강물']

In [76]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [77]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['강물', '처녀', '나무', '가문', '기도']

In [78]:
df['시'].iloc[0]

'우리가 물이 되어 만난다면\n가문 어느 집에선들 좋아하지 않으랴.\n우리가 키 큰 나무와 함께 서서\n우르르 우르르 비오는 소리로 흐른다면.\n\n흐르고 흘러서 저물녘엔\n저 혼자 깊어지는 강물에 누워\n죽은 나무 뿌리를 적시기도 한다면.\n아아, 아직 처녀인\n부끄러운 바다에 닿는다면.\n\n그러나 지금 우리는\n불로 만나려 한다.\n벌써 숯이 된 뼈 하나가\n세상에 불타는 것들을 쓰다듬고 있나니.\n\n만 리 밖에서 기다리는 그대여\n저 불 지난 뒤에 \n흐르는 물로 만나자.\n푸시시 푸시시 불 꺼지는 소리로 말하면서\n올 때는 인적 그친\n넓고 깨끗한 하늘로 오라.'